In [83]:
import json
import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt

from scipy.optimize import minimize
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression

In [84]:
data = pd.read_csv('returns.csv', index_col=0)
data = data[1:]
data.head()


,A,AAPL,ABBV,ABNB,ABT,ACGL,ACN,ADBE,ADI,ADM,...,WTW,WY,WYNN,XEL,XOM,XYL,YUM,ZBH,ZBRA,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
2000-01-04,-0.076389,-0.084310,NaN,NaN,-0.028572,-0.005435,NaN,-0.083889,-0.050589,-0.010416,...,NaN,-0.037568,NaN,0.023026,-0.019153,NaN,-0.020101,NaN,-0.014428,NaN
2000-01-05,-0.062030,0.014633,NaN,NaN,-0.001838,0.092896,NaN,0.019771,0.014598,-0.015790,...,NaN,0.052975,NaN,0.038585,0.054515,NaN,0.005129,NaN,0.019144,NaN
2000-01-06,-0.038077,-0.086539,NaN,NaN,0.034991,-0.010000,NaN,0.008163,-0.027337,0.005347,...,NaN,0.046778,NaN,-0.009288,0.051697,NaN,-0.008503,NaN,-0.054144,NaN
2000-01-07,0.083334,0.047369,NaN,NaN,0.010676,0.055556,NaN,0.048583,0.028106,0.015958,...,NaN,-0.032883,NaN,0.000000,-0.002934,NaN,-0.022299,NaN,-0.011098,NaN
2000-01-10,0.060577,-0.017588,NaN,NaN,-0.007042,0.033493,NaN,0.038610,0.087051,0.000000,...,NaN,-0.004359,NaN,0.000000,-0.013981,NaN,0.040350,NaN,0.033668,NaN


In [85]:
#fidn the winners and losers based on a 36 month window
data.index = pd.to_datetime(data.index)
monthly_ret_df = ((1 + data).resample("M").prod() - 1)
monthly_ret_df

,A,AAPL,ABBV,ABNB,ABT,ACGL,ACN,ADBE,ADI,ADM,...,WTW,WY,WYNN,XEL,XOM,XYL,YUM,ZBH,ZBRA,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
2000-01-31,-0.080730,-0.073144,0.000000,0.000000,-0.065070,0.271739,0.000000,-0.160153,0.036729,-0.020833,...,0.000000,-0.173207,0.000000,0.013158,0.058261,0.000000,-0.232831,0.000000,0.051054,0.000000
2000-02-29,0.569405,0.104820,0.000000,0.000000,0.013436,0.042735,0.000000,0.852441,0.681818,-0.140034,...,0.000000,-0.105665,0.000000,-0.087663,-0.086158,0.000000,-0.069869,0.000000,0.124604,0.000000
2000-03-31,0.001204,0.184841,0.000000,0.000000,0.066288,0.073771,0.000000,0.091566,0.023848,0.024845,...,0.000000,0.110841,0.000000,0.131673,0.035684,0.000000,0.166666,0.000000,-0.248826,0.000000
2000-04-30,-0.147836,-0.086516,0.000000,0.000000,0.097843,-0.060115,0.000000,0.086468,-0.045807,-0.036364,...,0.000000,-0.063596,0.000000,0.116965,-0.004006,0.000000,0.090544,0.000000,0.140000,0.000000
2000-05-31,-0.169253,-0.322922,0.000000,0.000000,0.058536,-0.025380,0.000000,-0.069251,0.002441,0.207111,...,0.000000,-0.063303,0.000000,0.014327,0.078226,0.000000,-0.134687,0.000000,-0.157895,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-08-31,-0.005748,-0.042384,-0.017516,-0.135620,-0.075721,-0.010683,0.023455,0.024113,-0.088959,-0.061648,...,-0.021625,-0.032910,-0.067278,-0.089271,0.045333,-0.078713,-0.055813,-0.137749,-0.106994,0.012866
2023-09-30,-0.076402,-0.088678,0.014289,0.043025,-0.058795,0.037080,-0.051456,-0.088390,-0.032224,-0.048928,...,0.014668,-0.063817,-0.088479,0.010616,0.057469,-0.120823,-0.034317,-0.055879,-0.139922,-0.086767
2023-10-31,-0.073704,-0.002570,-0.043384,-0.137891,-0.018344,0.087442,-0.028615,0.043460,-0.101433,-0.051047,...,0.128876,-0.064253,-0.050103,0.035827,-0.099762,0.027573,-0.032656,-0.069595,-0.114573,-0.095437


In [86]:
def rolling_36m_avg_and_rank(ret_df, window = 36, min_periods = 36):


    roll_avg = (
        ret_df
        .rolling(window=window, min_periods=min_periods)
        .mean()
    )

    def rank_row(row):
        mask = (row.notna()) & (row != 0)
        ranked = row[mask].rank(method='first', ascending=False)
        return ranked.reindex(row.index)

    rank_df = roll_avg.apply(rank_row, axis=1)

    return roll_avg, rank_df



In [87]:
roll_avg, rank = rolling_36m_avg_and_rank(monthly_ret_df, window=36, min_periods=36)
roll_avg = roll_avg[35:]
roll_avg.head(100)

,A,AAPL,ABBV,ABNB,ABT,ACGL,ACN,ADBE,ADI,ADM,...,WTW,WY,WYNN,XEL,XOM,XYL,YUM,ZBH,ZBRA,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
2002-12-31,-0.015859,-0.016192,0.0,0.0,0.008445,0.031417,0.010100,0.013257,0.004820,0.009852,...,0.017354,-0.002758,0.000237,0.003351,3.251858e-06,0.0,0.013447,0.010147,0.009136,0.0
2003-01-31,-0.015906,-0.014102,0.0,0.0,0.009104,0.023290,0.007877,0.019534,0.003870,0.009646,...,0.014816,0.001622,0.001593,0.003036,-2.243166e-03,0.0,0.018721,0.009799,0.007020,0.0
2003-02-28,-0.037251,-0.015757,0.0,0.0,0.006909,0.022776,0.005846,-0.003009,-0.008999,0.011011,...,0.013750,0.005598,0.002542,0.005647,2.321100e-04,0.0,0.021417,0.012096,0.007149,0.0
2003-03-31,-0.037390,-0.022501,0.0,0.0,0.006619,0.023108,0.006135,-0.002177,-0.011242,0.010066,...,0.016825,0.001394,0.004847,0.006298,2.361645e-07,0.0,0.017394,0.014749,0.014642,0.0
2003-04-30,-0.027220,-0.019941,0.0,0.0,0.006316,0.025441,0.007067,-0.001245,-0.004293,0.011796,...,0.020325,0.004415,0.007733,0.004588,3.102099e-04,0.0,0.015302,0.013761,0.011732,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2010-11-30,0.005728,0.023040,0.0,0.0,-0.001811,0.008188,0.010117,-0.004205,0.011020,0.000493,...,0.000044,-0.003044,0.021348,0.005617,-3.544638e-03,0.0,0.013180,-0.002821,0.002760,0.0
2010-12-31,0.011612,0.021641,0.0,0.0,-0.000318,0.007269,0.012250,-0.001616,0.011851,-0.006166,...,0.003913,0.000472,0.025342,0.006353,-3.534325e-03,0.0,0.011771,-0.000938,0.006729,0.0
2011-01-31,0.014058,0.031880,0.0,0.0,-0.001835,0.007289,0.015045,0.005503,0.015633,-0.002308,...,0.008452,0.008774,0.027977,0.008616,1.706592e-03,0.0,0.013491,-0.003137,0.010588,0.0


In [88]:
rank = rank[35:]
rank.head(100)

,A,AAPL,ABBV,ABNB,ABT,ACGL,ACN,ADBE,ADI,ADM,...,WTW,WY,WYNN,XEL,XOM,XYL,YUM,ZBH,ZBRA,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
2002-12-31,359.0,360.0,NaN,NaN,212.0,33.0,192.0,154.0,242.0,196.0,...,109.0,302.0,285.0,256.0,287.0,NaN,148.0,191.0,206.0,NaN
2003-01-31,358.0,354.0,NaN,NaN,202.0,62.0,211.0,83.0,252.0,199.0,...,130.0,269.0,270.0,257.0,300.0,NaN,90.0,195.0,216.0,NaN
2003-02-28,374.0,349.0,NaN,NaN,229.0,67.0,236.0,300.0,334.0,187.0,...,152.0,241.0,267.0,239.0,280.0,NaN,79.0,173.0,228.0,NaN
2003-03-31,375.0,363.0,NaN,NaN,201.0,48.0,208.0,280.0,333.0,178.0,...,99.0,254.0,224.0,206.0,264.0,NaN,95.0,119.0,120.0,NaN
2003-04-30,371.0,364.0,NaN,NaN,229.0,47.0,217.0,298.0,316.0,167.0,...,72.0,247.0,205.0,245.0,286.0,NaN,119.0,142.0,168.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2010-11-30,225.0,27.0,NaN,NaN,362.0,182.0,155.0,391.0,141.0,327.0,...,335.0,380.0,32.0,226.0,383.0,NaN,102.0,377.0,287.0,NaN
2010-12-31,158.0,44.0,NaN,NaN,367.0,232.0,144.0,383.0,154.0,416.0,...,302.0,358.0,32.0,252.0,394.0,NaN,155.0,376.0,243.0,NaN
2011-01-31,149.0,19.0,NaN,NaN,398.0,276.0,136.0,321.0,123.0,406.0,...,251.0,247.0,29.0,249.0,376.0,NaN,156.0,410.0,211.0,NaN


In [89]:
def build_long_short_weights(rank_df, top_pct = 0.1):

    weight_list = []

    for date, ranks in rank_df.iterrows():
        valid_ranks = ranks.dropna()

        if valid_ranks.empty:
            weight_list.append(pd.Series(0, index=ranks.index))
            continue

        n_assets = len(valid_ranks)
        n_long = max(1, int(n_assets * top_pct))
        n_short = max(1, int(n_assets * top_pct))

        long_assets = valid_ranks.nsmallest(n_long).index
        short_assets = valid_ranks.nlargest(n_short).index

        weights = pd.Series(0, index=ranks.index)

        if n_long > 0:
            weights.loc[long_assets] = 1.0 / n_long
        if n_short > 0:
            weights.loc[short_assets] = -1.0 / n_short

        weight_list.append(weights)

    weights_df = pd.DataFrame(weight_list, index=rank_df.index)
    return weights_df


weights_df = build_long_short_weights(rank)
weights_df = weights_df[35:]
weights_df.head(100)

,A,AAPL,ABBV,ABNB,ABT,ACGL,ACN,ADBE,ADI,ADM,...,WTW,WY,WYNN,XEL,XOM,XYL,YUM,ZBH,ZBRA,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
2005-11-30,0.0,0.025,0.0,0.0,-0.025,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.025,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
2005-12-31,0.0,0.025,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.025,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
2006-01-31,0.0,0.025,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.025,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
2006-02-28,0.0,0.025,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.025,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
2006-03-31,0.0,0.025,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.025,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2013-10-31,0.0,0.000,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,-0.021739
2013-11-30,0.0,0.000,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,-0.021739
2013-12-31,0.0,0.000,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,-0.021739


In [90]:
def evaluate_portfolio_holding_period_monthly(weights_df, monthly_returns, holding_period_months= 36):
    portfolio_total_returns = {}

    monthly_dates = monthly_returns.index

    for formation_date, weights in weights_df.iterrows():
        start_idx = monthly_dates.get_indexer([formation_date], method='nearest')[0]

        end_idx = start_idx + holding_period_months

        holding_period = monthly_returns.iloc[start_idx:end_idx]

        monthly_portfolio_returns = (holding_period * weights).sum(axis=1)

        total_return = (1 + monthly_portfolio_returns).prod() - 1

        portfolio_total_returns[formation_date] = total_return

    return pd.Series(portfolio_total_returns)

In [91]:
portfolio_returns = evaluate_portfolio_holding_period_monthly(weights_df, monthly_ret_df, 36)
portfolio_returns

2005-11-30    0.207130
2005-12-31    0.008309
2006-01-31    0.096521
2006-02-28    0.018510
2006-03-31    0.110810
                ...   
2023-08-31    0.036442
2023-09-30   -0.010671
2023-10-31   -0.057393
2023-11-30   -0.065940
2023-12-31   -0.028303
Length: 218, dtype: float64

In [92]:
total = portfolio_returns.mean()
total

0.12635132782583192